# Spherical Grid

In this example notebook, we will construct a grid for the unit sphere using spherical coordinates $r, \theta, \varphi$. Then we will define a function $f(r, \theta, \varphi$) and define and apply the Laplacian in spherical coordinates to $f$, namely

$$
\nabla^2 f=\frac{\partial^2 f}{\partial r^2}+\frac{2}{r} \frac{\partial f}{\partial r}+\frac{1}{r^2 \sin \theta} \frac{\partial}{\partial \theta}\left(\sin \theta \frac{\partial f}{\partial \theta}\right)+\frac{1}{r^2 \sin ^2 \theta} \frac{\partial^2 f}{\partial \varphi^2}
$$

As you can see, there are a few sensitive points that we have to take care of: the coordinates $r=0$ and $\theta = 0$ or $\pi$ would lead to zeros in the denominator. So we cannot sample the grid on the center of the sphere and also not at the north and south poles. Instead, we will use points nearby by using a cutoff parameter.

In [1]:
from numgrids import *
import numpy as np

In [2]:
cut_off = 1.E-3

In [3]:
grid = Grid(
    Axis(AxisType.CHEBYSHEV, 30, cut_off, 1),  # radial axis
    Axis(AxisType.CHEBYSHEV, 30, cut_off, np.pi - cut_off), # polar axis
    Axis(AxisType.EQUIDISTANT_PERIODIC, 50, 0, 2*np.pi),  # azimuthal axis
)

So this is the grid. Now construct the Laplacian:

In [4]:
R, Theta, Phi = grid.meshed_coords

In [5]:
dr2 = Diff(grid, 2, 0)
dr = Diff(grid, 1, 0)
dtheta = Diff(grid, 1, 1)
dphi2 = Diff(grid, 2, 2)

def laplacian(f):
    
    return dr2(f) + 2 / R * dr(f) + \
            1/(R**2 * np.sin(Theta)) * dtheta(np.sin(Theta) * dtheta(f)) + \
            1/(R**2 * np.sin(Theta)**2) * dphi2(f)

Define the function on the grid. For simplicity we will use $f(r, \theta, \varphi) = r^2$ because its
Laplacian is simply 6 at every point of the sphere, so that we can easily compare our numerical result with the exact result:

In [6]:
f = R**2

In [7]:
lap_f = laplacian(f)

In [8]:
np.max(np.abs(lap_f - 6))

3.91352052986349e-08

Not too bad, after all.